<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Post Processing

This notebook gathers all the diagrams that can be used to analyze one or several aircrafts, allowing for example to compare modified architectures with respect to a reference one.

The postprocessing functions all use the generated xml files of the analysis mode or from a simple MDA. The user can manually write the name of each file in the "name" parameter of the functions. This name will appear in the legend of the diagram. To add an aircraft to a figure, the user simply has to call the function with the desired aircraft file by using the same figure as for the previous lines of command :

*fig = plot_function(aircraft_file, name="", parameters, fig=fig)*.

This works for most functions.

List of the diagrams :

**GEOMETRY**
- Top View of the Aircraft Geometry

**AERODYNAMICS**
- Distribution of the Lift Coefficient along the Wing (requires two xml files to work since it aims to directly compare the lift distributions)
- Drag Breakdown
- Non Equilibrated Aircraft Drag Polar
- Equilibrated Aircraft Drag Polar

**WEIGHT/BALANCE**
- Course of the barycenter with the static margins highlighted
- Mass Breakdown

**PERFORMANCES**
- Payload Range
- Evolution Diagram (Maneuver + Gust)

**LOAD ANALYSIS**
- Force Distribution on the Wing
- Shear Force on the Wing
- Root Bending Moments on the Wing
- Bending Moments on the Fuselage

## Initialization of work environment

The user has to choose here the files he wants to postprocess. He can take mda output files (in the data folder), or output files from the analysis mode (in the output folder). Since the theoretical amount of files is infinite no widget or approach has been implemented to render the choice of file user-friendly. The user has to make sure that the files he wants to analyze are processed in this cell. Once it is done their path name can be used as the input of the functions.

In [ ]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, "data")
WORK_FOLDER_PATH = pth.join(current_path, "workdir")
OUTPUT_FOLDER_PATH = pth.join(current_path, "output")

OUTPUT_REF_FILE = pth.join(OUTPUT_FOLDER_PATH, "output_ref.xml")
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, "output_mod_fuselage_tbm.xml")

# MDA Output Files
BEECH_MDA = pth.join(DATA_FOLDER_PATH, "output_mda_beech.xml")
CIRRUS_MDA = pth.join(DATA_FOLDER_PATH, "output_mda_cirrus_sr22.xml")
TBM_MDA = pth.join(DATA_FOLDER_PATH, "output_mda_tbm930.xml")

# For using all screen width
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
api_cs25.variable_viewer(BEECH_MDA)

In [ ]:
api_cs25.variable_viewer(OUTPUT_MOD_FILE)

## GEOMETRY

### Top View of the Aircraft Geometry

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

fig = aircraft_geometry_plot(TBM_MDA, name="TBM 930", plot_nacelle=True)
fig = aircraft_geometry_plot(
    OUTPUT_MOD_FILE, name="TBM 930 with fuselage stretched by 1 meter", fig=fig, plot_nacelle=False
)
fig.show()

## AERODYNAMICS

### Lift coefficient distribution along the wing

prop_on = True : takes in account the influence of the propeller on the airflow.

prop_on = False : does not take in account the influence of the propeller on the airflow.

For aircrafts with engines mounted under the wing we can clearly see the impact of the propeller.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import cl_wing_diagram


prop_on = True

[fig, fig1] = cl_wing_diagram(
    TBM_MDA, OUTPUT_MOD_FILE, prop_on, name_ref="reference aircraft", name_mod="modified aircraft"
)
fig.show()
fig1.show()

### Drag breakdown Diagram

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import drag_breakdown_diagram

fig = drag_breakdown_diagram(OUTPUT_MDA_FILE)
fig.show()

### Non Equilibrated Drag Polar
The value displayed on the curves is the maximum ratio of the CL over the CD ie the finesse.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_polar

fig = aircraft_polar(BEECH_MDA, name="reference aircraft", equilibrated=False)
# fig = aircraft_polar(OUTPUT_MOD_FILE, name='modified aircraft', equilibrated=False, fig=fig)
fig.show()

### Equilibrated Aircraft Drag Polar

The range of (CD, CL) points is obtained with the theoretical variation of the aircraft MTOW up to the mass where the CL0_clean is reached.

The displayed value of finesse for the equilibrated polar is too low. The quantity for the non-equilibrated drag polars is preferably used.

In [ ]:
api_cs25.variable_viewer(TBM_MDA)

In [ ]:
from fastga.models.aerodynamics.components.compute_polar import _compute_equilibrated_polar
from fastga.utils.postprocessing.analysis_and_plots import aircraft_polar

TEMP_FILE = pth.join(WORK_FOLDER_PATH, "temp.xml")
shutil.copy(BEECH_MDA, TEMP_FILE)


def polar_variation(cg_ratio):
    compute_polar_cruise = api_cs23.generate_block_analysis(
        _compute_equilibrated_polar(low_speed_aero=False, x_cg_ratio=cg_ratio),
        [],
        str(TEMP_FILE),
        True,
    )

    output = compute_polar_cruise({})

    compute_polar_low_speed = api_cs23.generate_block_analysis(
        _compute_equilibrated_polar(low_speed_aero=True, x_cg_ratio=cg_ratio),
        [],
        str(TEMP_FILE),
        True,
    )

    output = compute_polar_low_speed({})

    fig = aircraft_polar(TEMP_FILE, name="Beech 76", equilibrated=True)
    fig.show()

In [ ]:
from ipywidgets import widgets, Layout
from fastoad.io import VariableIO
from IPython.display import display, clear_output

variables = VariableIO(TEMP_FILE).read()
cg_aft = round(variables["data:weight:aircraft:CG:aft:MAC_position"].value[0], 2)
cg_fwd = round(variables["data:weight:aircraft:CG:fwd:MAC_position"].value[0], 2)

cg_choice = widgets.FloatSlider(
    value=cg_fwd,
    min=cg_fwd,
    max=cg_aft,
    step=0.01,
    description="CG position (ratio of MAC):",
    style={"description_width": "initial"},
    layout=Layout(width="30%"),
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)


def eventhandler(change):
    if change.new:
        clear_output()
        display(cg_choice)
        polar_variation(cg_choice.value)


cg_choice.observe(eventhandler, names="value")
display(cg_choice)

## WEIGHT / BALANCE

### Course of the barycenter

"empty CG" refers to the aircraft barycenter without payload and without fuel.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import cg_lateral_diagram


fig = cg_lateral_diagram(TBM_MDA, name="TBM 930", color="MediumPurple")
fig = cg_lateral_diagram(
    OUTPUT_MOD_FILE, name="TBM 930 with fuselage stretched by 1 meter", fig=fig, color="LightSalmon"
)
fig.show()

### Mass breakdown

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import mass_breakdown_bar_plot

fig = mass_breakdown_bar_plot(TBM_MDA, name="TBM 930")
fig = mass_breakdown_bar_plot(OUTPUT_MOD_FILE, name="TBM 930 + 15 % wing span", fig=fig)
fig.show()

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import mass_breakdown_sun_plot

fig = mass_breakdown_sun_plot(BEECH)
# fig = mass_breakdown_sun_plot(OUTPUT_MOD_FILE, name='modified aircraft', fig=fig)
fig.show()

## PERFORMANCES

### Evolution Diagram

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import evolution_diagram


fig = evolution_diagram(TBM_MDA, name="TBM 930")
# fig = evolution_diagram(OUTPUT_MOD_FILE,name='modified aircraft',fig=fig)
fig.show()

### Payload Range Diagram
It can be necessary to manually compute the payload range since this process is not automatically done in the analysis because of its rather high computational time (1-2 min).

In [ ]:
from fastga.models.performances.payload_range import ComputePayloadRange

# engine_id = "fastga.wrapper.propulsion.basicIC_engine"
engine_id = "fastga.wrapper.propulsion.basic_turboprop"

compute_payload_range = api_cs23.generate_block_analysis(
    ComputePayloadRange(
        propulsion_id=engine_id,
    ),
    [],
    str(TBM_MDA),
    True,
)
output = compute_payload_range({})

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import payload_range

fig = payload_range(TBM_MDA, name="Beechcraft Model 76 Duchess")
# fig = payload_range(OUTPUT_MOD_FILE, name='modified aircraft', fig=fig)
fig.show()

## LOAD ANALYSIS

### Force Distribution on the Wing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import (
    force_repartition_diagram,
)

fig = force_repartition_diagram(OUTPUT_REF_FILE, name="reference aircraft")
fig.show()

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import (
    force_repartition_diagram,
)

fig = force_repartition_diagram(OUTPUT_MOD_FILE, name="modified aircraft")
fig.show()

### Shear Force on the Wing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import shear_diagram

fig2 = shear_diagram(OUTPUT_REF_FILE, name="reference aircraft")
fig2.show()

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import shear_diagram

fig2 = shear_diagram(OUTPUT_MOD_FILE, name="modified aircraft")
fig2.show()

### Root Bending Moments on the Wing

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import rbm_diagram

fig3 = rbm_diagram(OUTPUT_REF_FILE, name="reference aircraft")
fig3.show()

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import rbm_diagram

fig3 = rbm_diagram(OUTPUT_MOD_FILE, name="modified aircraft")
fig3.show()

### Bending Moments on the Fuselage

In [ ]:
from fastga.utils.postprocessing.load_analysis.analysis_and_plots_la import fuselage_bending_diagram

fig = fuselage_bending_diagram(TBM_MDA, name="reference aircraft")
fig = fuselage_bending_diagram(OUTPUT_MOD_FILE, name="modified aircraft", fig=fig)
fig.show()